In [131]:
public enum SignalType{
    Undefined,
    Buy,
    Sell
}

public enum PivotType { 
    Undefined,
    Low,
    High 
}

public record PriceInfo(DateTime Date,
    double Open,
    double High,
    double Low,
    double Close,
    double ChannelLow,
    double ChannelHigh,
    PivotType PivotType,
    double PivotValue,
    SignalType BreakoutAction,
    double TradeCapital);

In [132]:
using System.Globalization;
using System.IO;

List<PriceInfo> ParseCsv(string filePath)
{
    var records = new List<PriceInfo>();
    // Assuming the first line of the CSV contains headers and each subsequent line is a record.
    foreach (var line in File.ReadAllLines(filePath).Skip(1))
    {
        var columns = line.Split(',');

        var pivotType = string.IsNullOrWhiteSpace(columns[7]) 
            ? PivotType.Undefined 
            : Enum.Parse<PivotType>(columns[7]);

        double pivotValue = pivotType == PivotType.Undefined 
            ? 0 
            : double.Parse(columns[8]);
        var record = new PriceInfo
        (
            Date: DateTime.ParseExact(columns[0], "yyyy-MM-dd HH:mm:ss", CultureInfo.InvariantCulture),
            Open: double.Parse(columns[1]),
            High: double.Parse(columns[2]),
            Low: double.Parse(columns[3]),
            Close: double.Parse(columns[4]),
            ChannelLow: double.Parse(columns[5]),
            ChannelHigh: double.Parse(columns[6]),
            PivotType: pivotType,
            PivotValue: pivotValue,
            BreakoutAction: string.IsNullOrWhiteSpace(columns[9]) 
                ? SignalType.Undefined 
                : Enum.Parse<SignalType>(columns[9]),
            TradeCapital: double.Parse(columns[10])
        );
        records.Add(record);
    }
    return records;
}

In [133]:
#r "nuget: XPlot.Plotly, 4.0.6"
using XPlot.Plotly;



var targetFile = Environment.CurrentDirectory + "/breakout.csv";
var records = ParseCsv(targetFile);




Installed Packages XPlot.Plotly, 4.0.6

In [134]:
Trace GetPivotsLows(List<PriceInfo> records)
{
    var pivotPointsLow = records.Where(r => r.PivotType == PivotType.Low).ToList();
    return new Scatter
    {
        x = pivotPointsLow.Select(r => r.Date.ToString("yyyy-MM-dd HH:mm:ss")).ToArray(),
        y = pivotPointsLow.Select(r => r.PivotValue).ToArray(),
        mode = "markers",
        marker = new Marker
        {
            color = "Red",
            size = 10
        },
        name = "Pivot Low"
    };
}

In [135]:
Trace GetPivotHighs(List<PriceInfo> records)
{
    var pivotPointsHigh = records.Where(r => r.PivotType == PivotType.High).ToList();
    return new Scatter
    {
        x = pivotPointsHigh.Select(r => r.Date.ToString("yyyy-MM-dd HH:mm:ss")).ToArray(),
        y = pivotPointsHigh.Select(r => r.PivotValue).ToArray(),
        mode = "markers",
        marker = new Marker
        {
            color = "Green",
            size = 10
        },
        name = "Pivot High"
    };
}

In [136]:
Trace GetCandleStickTrace(List<PriceInfo> records)
{
    return new Candlestick
    {
        x = records.Select(r => r.Date.ToString("yyyy-MM-dd HH:mm:ss")).ToArray(),
        open = records.Select(r => r.Open).ToArray(),
        high = records.Select(r => r.High).ToArray(),
        low = records.Select(r => r.Low).ToArray(),
        close = records.Select(r => r.Close).ToArray(),
        name = "Candlestick"
    };
}

In [137]:
Trace GetChannelLowTrace(List<PriceInfo> records)
{
    return new Scatter
    {
        x = records.Select(r => r.Date.ToString("yyyy-MM-dd HH:mm:ss")).ToArray(),
        y = records.Select(r => r.ChannelLow).ToArray(),
        mode = "lines",
        line = new Line
        {
            color = "orange",
            width = 1
        },
        name = "Channel Low"
    };
}

In [138]:
Trace GetChannelHighTrace(List<PriceInfo> records)
{
    return new Scatter
    {
        x = records.Select(r => r.Date.ToString("yyyy-MM-dd HH:mm:ss")).ToArray(),
        y = records.Select(r => r.ChannelHigh).ToArray(),
        mode = "lines",
        line = new Line
        {
            color = "blue",
            width = 1
        },
        name = "Channel High"
    };
}

In [139]:
Trace GetBreakoutActionsTrace(List<PriceInfo> records)
{
    var breakoutActions = records.Select((r, index) => new { Date = r.Date, Action = r.BreakoutAction, Index = index })
                                 .Where(r => r.Action != SignalType.Undefined) // Assuming BreakoutAction is of SignalType enum
                                 .ToList();

    return new Scatter
    {
        x = breakoutActions.Select(r => r.Date.ToString("yyyy-MM-dd HH:mm:ss")).ToArray(),
        y = breakoutActions.Select(r => records[r.Index].Close).ToArray(), // Position at close price for visual context
        mode = "markers+text",
        marker = new Marker
        {
            color = breakoutActions.Select(r => r.Action == SignalType.Buy ? "Green" : r.Action == SignalType.Sell ? "Red" : "Grey").ToArray(),
            size = 10
        },
        text = breakoutActions.Select(r => r.Action.ToString()).ToArray(),
        textposition = "top center",
        name = "Breakout Actions"
    };
}



In [140]:

var candlestickData = GetCandleStickTrace(records);
var pivotHigh = GetPivotHighs(records);
var pivotLow = GetPivotsLows(records);
var channelLow = GetChannelLowTrace(records.TakeLast(100).ToList());
var channelHigh = GetChannelHighTrace(records.TakeLast(100).ToList());
var breakoutActions = GetBreakoutActionsTrace(records);

var chart = Chart.Plot(new Trace[] { candlestickData, pivotHigh, pivotLow, channelLow, channelHigh, breakoutActions});
chart.WithLayout(new Layout.Layout
{
    title = "Breakout Analysis",
    xaxis = new Xaxis { title = "Date" },
    yaxis = new Yaxis { title = "Price" }
});

chart.Show();